Checked 27.10.2023 G.Paaß
# Time Series Analysis with RNN using tf.data

This notebook uses code from [here](https://www.tensorflow.org/tutorials/structured_data/time_series). It uses Keras, which is now part of Tensorflow.

Prediction Task: **Time Series Analysis**
* predict next value of a one-dimensional or multidimensional time series.
* Dataset: A set of 400000 captions for images.

This model can be used to compute the probability of a sequence, as well as generate new sequences.


In [ ]:
import sys, os;
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys
import math
import json
import tensorflow as tf
import pandas as pd
import datetime
import seaborn as sns

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
!nvidia-smi

## Wheather Dataset

This tutorial uses a [weather time series dataset](https://www.bgc-jena.mpg.de/wetter/) recorded by the Max Planck Institute for Biogeochemistry.

This dataset contains 14 different features such as air temperature, atmospheric pressure, and humidity. These were collected every 10 minutes, beginning in 2003. For efficiency, you will use only the data collected between 2009 and 2016. This section of the dataset was prepared by François Chollet for his book Deep Learning with Python.

In [ ]:
# download 'jena_climate_2009_2016.csv.zip' to directory '~/.keras/datasets/''
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path) # remove extension '.zip' from path
csv_path

In [ ]:
os.path.splitext(zip_path)    # remove suffix from path
df = pd.read_csv(csv_path)    # Read a comma-separated values (csv) file into DataFrame.
df.head()                     # print with pandas-formatting

In [ ]:
df.columns

Here are the actual **variables** of the data:

1. Date Time

1. p (mbar) atmospheric pressure

1. T (degC) temperature

1. Tpot (K) potential temperature

1. Tdew (degC) dew point temperature

1. rh (%) relative humidity

1. VPmax (mbar) saturation water vapor pressure

1. VPact (mbar) actual water vapor pressure

1. VPdef (mbar) water vapor pressure deficit

1. sh (g/kg) specific humidity

1. H2OC (mmol/mol) water vapor concentration

1. rho (g/m3) air density

1. wv (m/s) wind velocity

1. max. wv (m/s) maximum wind velocity

1. wd (deg) wind direction

As you can see above, an observation is recorded every 10 minutes. This means that, for a single hour, you will have 6 observations. Similarly, a single day will contain 144 (6x24) observations.

This tutorial will just deal with **hourly predictions**, so start by sub-sampling the data from 10-minute intervals to one-hour intervals:

In [ ]:
df = pd.read_csv(csv_path)
# Slice [start:stop:step], starting from index 5 take every 6th record.
df = df[5::6]

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

Here is the evolution of a few features over time:
* over the whole data period
* over the first 480 measurements

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

## Data Preparation

Data Statistics

In [ ]:
df.describe().transpose()

The **correlation matrix** shows that some of the variables are highly correlated.

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)

### Wind velocity

One thing that should stand out is the min value of the wind velocity (wv (m/s)) and the maximum value (max. wv (m/s)) columns. This -9999 is likely erroneous.

There's a separate wind direction column, so the velocity should be greater than zero (>=0). Replace it with zeros:

In [ ]:
wv = df['wv (m/s)']
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

max_wv = df['max. wv (m/s)']
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

# The above inplace edits are reflected in the DataFrame.
df['wv (m/s)'].min()

### Wind Variable

The last column of the data, `wd (deg)`—gives the wind direction in units of degrees. Angles do not make good model inputs: 360° and 0° should be close to each other and wrap around smoothly. Direction shouldn't matter if the wind is not blowing.

But this will be easier for the model to interpret if you convert the wind direction and velocity columns to a wind **vector**:



In [ ]:
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

# Convert to radians.
wd_rad = df.pop('wd (deg)')*np.pi / 180

# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

The distribution of wind vectors is much simpler for the model to correctly interpret:

In [ ]:
plt.hist2d(df['Wx'], df['Wy'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')

#### Time

Similarly, the `Date Time` column is very useful, but not in this string form. Start by converting it to seconds:

In [ ]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)

Similar to the wind direction, the time in seconds is not a useful model input. Being weather data, it has clear daily and yearly periodicity. There are many ways you could deal with periodicity.

You can get usable signals by using sine and cosine transforms to clear "Time of day" and "Time of year" signals:

In [ ]:
day = 24*60*60           # number of secods of a day
year = (365.2425)*day    # number of seconds of a year

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
plt.plot(np.array(df['Day sin'])[:25])
plt.plot(np.array(df['Day cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

This gives the model access to the most important frequency features. In this case you knew ahead of time which frequencies were important.

If you don't have that information, you can determine which frequencies are important by extracting features with <a href="https://en.wikipedia.org/wiki/Fast_Fourier_transform" class="external">Fast Fourier Transform</a>. To check the assumptions, here is the `tf.signal.rfft` of the temperature over time. Note the **obvious peaks at frequencies near `1/year` and `1/day`**:

In [ ]:
fft = tf.signal.rfft(df['T (degC)'])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(df['T (degC)'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

As can be seen in the **correlation plot** windspeed, direction, and time is now correlated with the other variables.

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)



### Defining  time series lag
Given a specific time, let's say you want to predict the temperature 6 hours in the future. In order to make this prediction, you choose to use 5 days of observations. Thus, you would create a **window** containing the last 720(5x144) observations to train the model. Many such configurations are possible, making this dataset a good one to experiment with.

The first 300,000 rows of the data will be the training dataset, and there remaining will be the validation dataset. This amounts to ~2100 days worth of training data.

Setting seed to ensure reproducibility.

In [ ]:
print("df.shape",df.shape)
TRAIN_SPLIT = round(df.shape[0]*0.8)   # first 80% are used as training data
BATCH_SIZE = 256
BUFFER_SIZE = 10000
EVALUATION_INTERVAL = 200
EPOCHS = 20
tf.random.set_seed(13)

`plot_losses(history, title, ymax=-1)` and `plot_metric`

In [ ]:
# @title
def plot_losses(history, title, ymax=-1):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    #val_loss = np.array(val_loss)*uni_train_std # expand to degree C
    #loss = np.array(loss)*uni_train_std

    epochs = range(len(loss))

    plt.figure()
    plt.xlabel("epoch")
    plt.ylabel("loss value")
    if ymax>0:
        plt.ylim(0.0,ymax)  # limit y-range

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

def plot_metric(history, title, ymax=-1, metric="mean_absolute_error", ylabel="mean absolute error in degree C"):
    metr = history.history[metric]
    val_metr= history.history['val_'+metric]

    #val_loss = np.array(val_loss)*uni_train_std # expand to degree C
    #loss = np.array(loss)*uni_train_std

    epochs = range(len(metr))

    plt.figure()
    plt.xlabel("epoch")
    plt.ylabel(ylabel)
    if ymax>0:
        plt.ylim(0.0,ymax)  # limit y-range

    plt.plot(epochs, metr, 'b', label='Training '+metric)
    plt.plot(epochs, val_metr, 'r', label='Validation '+metric)
    plt.title(title)
    plt.legend()

    plt.show()


In [ ]:
# @title
def create_time_steps(length):
    return list(range(-length, 0))

def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt


## Forecasting Multivariate Time Series
### Defining Input Features
The original dataset contains fourteen features. For simplicity, this section considers only three of the original fourteen. The features used are air temperature, atmospheric pressure, and air density.

To use more features, add their names to this list.

`'Date Time', 'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)',
       'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)'`

In [ ]:
df.columns

In [ ]:
features_considered = ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
                        'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
                        'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy',
                        'Day sin', 'Day cos', 'Year sin', 'Year cos']
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']
#features_considered = ['T (degC)', 'T (degC)']                  # only 1 variable

name2id = {}
for i in range(len(features_considered)):
  name2id[features_considered[i]] = i
print("name2id = ",name2id,"\n")

featDat = df[features_considered]
#featDat.index = df['Date Time']
featDat.head()

In [ ]:
name2id

#### Plotting Input Features

In [ ]:
featDat.plot(subplots=True)

### Preprocessing Data
As mentioned, the first step will be to **standardize** the dataset using the mean and standard deviation of the training data.

In [ ]:
dataset = featDat.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
print("data_mean",data_mean)
print(" data_std",data_std)

dataset = (dataset-data_mean)/data_std

#### Creating Input and Output Data
In a **single step** setup, the model learns to predict a single point in the future based on some history provided.

The below function performs the same windowing task as below, however, here it samples the past observation based on the step size given.

In [ ]:
def multivariate_data(dataset,      # dataset of input variables
                      target,       # dataset of output variables
                      start_index,  # first index to use
                      end_index,    # end index to use
                      history_size,
                      target_size,
                      step,
                      single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In this tutorial, the network is shown data from the last five (5) days, i.e. 720 observations that are sampled every hour. The sampling is done every one hour since a drastic change is not expected within 60 minutes. Thus, 120 observation represent history of the last five days. For the single step prediction model, the label for a datapoint is the temperature **12 hours into the future**. In order to create a label for this, the temperature after 72(12*6) observations is used.

In [ ]:
models = []   # list to store description and model results: each model yields a dictionary

In [ ]:
past_history = 120   # use 120 hours as input
future_target = 12   # predict value at t+1+future_target ( must be >= 0)
past_history = 24    # use 24 hours as input
future_target = 0    # predict value at t+1+future_target ( must be >= 0)
STEP = 1             # use only elements i, i+STEP, i+2STEP ...
ivpred = [name2id['T (degC)']]     # variables to be predicted

x_train_single, y_train_single = multivariate_data(dataset,
                                                   dataset[:, ivpred],
                                                   0,
                                                   TRAIN_SPLIT,
                                                   past_history,
                                                   future_target,
                                                   STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset,
                                               dataset[:, ivpred],
                                               TRAIN_SPLIT,  # first index to use
                                               None,         # to end o dataset
                                               past_history,
                                               future_target,
                                               STEP,
                                               single_step=True)

print ('Single window of past history : {}'.format(x_train_single[0].shape))
print('Single input Data',x_train_single[0])
print('Single data to be predicted',y_train_single[0])
print("----- Compare with full data -----")
dataset[:(past_history+STEP+future_target+2),:]

In [ ]:
# Create tf.data.Dataset for faster execution.
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [ ]:
mdict = {'past_history': past_history, 'future_target' :  future_target,
        'STEP': STEP,'ivpred' : ivpred, 'features_considered':features_considered}

### Defining the Multilayer Network model

In [ ]:
dropout=0.0
nhid = [32,32]    # length of hidden vector in each layer
patience = 4      # for early stopping

mdict.update({"type":'MLP', 'dropout':dropout, 'nhid':nhid, 'patience':patience})

input_dim = x_train_single.shape
nlayer= len(nhid)+1

model1 = tf.keras.models.Sequential()

# ------ LAYER 1 ----------
model1.add(tf.keras.layers.Flatten(input_shape=input_dim[1:])) # convert input matrix to vector x

if nlayer>1:
  for ilayer in range(nlayer-1):
    ### these LSTM layers feed their output to the next LSTM layer
    model1.add(tf.keras.layers.Dense(nhid[ilayer],
                                     activation = 'relu'))
    if dropout>0:
      model1.add(tf.keras.Dropout(dropout))

### this layer generates the output variable(s) with linear activation
model1.add(tf.keras.layers.Dense(len(ivpred)))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=patience,
                                                  mode='min')

model1.compile(optimizer=tf.keras.optimizers.RMSprop(),
                          metrics=[tf.keras.metrics.MeanAbsoluteError()],
                          loss='mae')   # mae mean absolute error, or mse mean square error
mdict

### Defining the LSTM model

In [ ]:
dropout=0.0
nhid = [32,32]   # length of hidden vector in each layer
patience = 3   # for early stopping

nlayer = len(nhid)+1

mdict.update({"type":'LSTM', 'dropout':dropout, 'nhid':nhid, 'patience':patience})

model1 = tf.keras.models.Sequential()
if nlayer>2:
  for ilayer in range(nlayer-1):
    ### these LSTM layers feed their output to the next LSTM layer
    model1.add(tf.keras.layers.LSTM(32,
                                    dropout=dropout,
                                    return_sequences=True,
                                    input_shape=x_train_single.shape[-2:]))
### this LSTM layer just returns the hidden vector at the end
model1.add(tf.keras.layers.LSTM(32, dropout=dropout))
### this layer generates the output variable(s)
model1.add(tf.keras.layers.Dense(len(ivpred)))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

model1.compile(optimizer=tf.keras.optimizers.RMSprop(),
                          metrics=[tf.keras.metrics.MeanAbsoluteError()],
                          loss='mae')   # mae mean absolute error, or mse mean square error
mdict

### List the Model Architecture

Let's check out a sample prediction.

In [ ]:
for x, y in val_data_single.take(1):  # take a single element from validation
    print("shape of a prediction",model1.predict(x).shape)
                                      # initializes parameters
model1.summary()

### Train the Model

In [ ]:
model1.fit?

In [ ]:
validation_steps = int(len(x_val_single)/BATCH_SIZE)  # number of batches in validation data
model1_history = model1.fit(train_data_single,
                            epochs=EPOCHS,
                            steps_per_epoch=EVALUATION_INTERVAL,
                            validation_data=val_data_single,
                            validation_steps=validation_steps,
                            callbacks=[early_stopping])
mdict.update({'model':model1, 'history':model1_history})


In [ ]:
mdict

#### Evaluate

In [ ]:
plot_losses(model1_history, 'Single Step Training and validation loss', ymax=-1)

In [ ]:
plot_metric(model1_history, 'Single Step Training and validation mean_absolute_error',
            ymax=-1, metric="mean_absolute_error", ylabel="mean absolute error in degree C")

In [ ]:
result = model1.evaluate(val_data_single,steps=50)
print(model1.metrics_names,result)
mdict.update({model1.metrics_names[0]:result[0], model1.metrics_names[1]:result[1]})

In [ ]:
models.append(mdict)

In [ ]:
mdict

In [ ]:
models

1 layer LSTM, time series with 1 input:  ['loss'] 0.24583026751875878
1 layer LSTM, time series with 3 inputs: ['loss'] 0.2547184965014458
2 layer LSTM, time series with 3 inputs: ['loss'] 0.2556210482120514

In [ ]:
models

#### Predict a single step future
Now that the model is trained, let's make a few sample predictions. The model is given the history of three features over the past five days sampled every hour (120 data-points), since the goal is to predict the temperature, the plot only displays the past temperature. The prediction is made one day into the future (hence the gap between the history and prediction).

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))

def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt

In [ ]:
for x, y in val_data_single.take(5):  # take these elements from validatio set
    pred = model1.predict(x)[0]
    inp = x[0]
    plot = show_plot([inp[:,ivpred].numpy(),  # lagged values of the output var in the input
                      y[0].numpy(),           # observed output value
                      pred],                  # predicted value
                      future_target,          # how many steps in the future
                     'Predicting '+str(future_target)+" steps from "+str(past_history)
                     + " past 3-dimensional inputs" )
    plot.show()

<font color='blue'>**Task 1:**</font>   Investigate a models, where the task is to predict 12 hours into future: `future_target=12`.


Try to improve the LSTM model and the MLP model such that the results get better. <br>
Modify `features_considered`, number of hidden variables, number of layers, dropout, etc.

<font color='blue'>**Task 2:**</font>   Modify the code such that several variables can be predicted simultaneously.